Install requirements

In [54]:
! pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using huggingface transformers pipeline to generate text

In [55]:
from transformers import pipeline
import torch
torch.manual_seed(42)
model_type = "openai-community/gpt2"
device = "cuda" if torch.cuda.is_available() else "cpu"
generator = pipeline("text-generation", model=model_type,device=device)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=1,truncation=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Hello, I\'m a language model, so you can\'t define something in any other language. Let me introduce another topic:\n\nThe name "'}]

Split the pipeline into 2 parts
1. Encode the text into tokens
2. forward pass of the model
3. decode the tokens into text
![image](images/tokenizer.png)

In [56]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# Encode the text into tokens
tokenizer = AutoTokenizer.from_pretrained(model_type,use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_type).to(device)
text = 'Hello, I am a language model,'
tokens = tokenizer(text, return_tensors='pt').to(device)
tokens

{'input_ids': tensor([[15496,    11,   314,   716,   257,  3303,  2746,    11]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [57]:
# generation of the model
max_length = 30
input_ids, attention_mask = tokens['input_ids'], tokens['attention_mask']
generation = model.generate(input_ids=input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id,do_sample=True)
generation.shape[1] == max_length

True

In [58]:
# decode the tokens into text
text = tokenizer.decode(generation[0], skip_special_tokens=True)
text

'Hello, I am a language model, a language design framework, and a language for language engineering, and also a language for analysis. Today, I'